<a href="https://colab.research.google.com/github/kashyap333/Playground-Series-Season-3-Episode-4./blob/main/Playground_Series_Season_3%2C_Episode_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c playground-series-s3e4
!unzip playground-series-s3e4

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [130]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.isna().values.any()

In [ ]:
train.Class.unique()

In [ ]:
sns.countplot(x='Class', data=train)

In [131]:
train.drop(['id', 'Time'], axis=1, inplace = True)
test.drop(['id', 'Time'], axis=1, inplace = True)

In [133]:
X, y = train_test_split(train, test_size=0.2, random_state=42,  stratify=train.Class)

In [ ]:
X.Class.value_counts(), y.Class.value_counts()

In [123]:
SL = StandardScaler()

In [135]:
X_train = X.drop('Class', axis=1)
X_test = y.drop('Class', axis=1)
y_train = X['Class']
y_test = y['Class']

X_train[X_train.columns] = SL.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = SL.fit_transform(X_test[X_test.columns])

In [101]:
'''sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)
X_test, y_test = sm.fit_resample(X_test, y_test)
sns.countplot('Class', data=pd.DataFrame(y_train))'''

In [ ]:
plt.figure(figsize=(15,30))
for i in enumerate(train.drop(['id', 'Time', 'Class'], axis=1).columns):
  plt.subplot(6, 5, i[0]+1)
  sns.histplot(data=train, x=i[1], hue='Class')


In [137]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold

In [138]:
xgb_params = {'n_estimators'     : 2000,
              'min_child_weight' : 96,
              'max_depth'        : 7,
              'learning_rate'    : 0.18,
              'subsample'        : 0.95,
              'colsample_bytree' : 0.95,
              'reg_lambda'       : 1.50,
              'reg_alpha'        : 1.50,
              'gamma'            : 1.50,
              'max_bin'          : 512,
              'random_state'     : 42,
              'objective'        : 'binary:logistic',
              'tree_method'      : 'hist',
              'eval_metric'      : 'rmse',
             }

In [ ]:
xgbc = XGBClassifier(**xgb_params)
xgbc.fit(X_train, y_train)
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_train, y_train, cv=kfold)
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_test, y_test, cv=kfold)
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
test.drop(['id', 'Time'], axis=1, inplace = True)
test_pred = xgbc.predict_proba(test)

In [ ]:
test = pd.read_csv('test.csv')
submission = pd.DataFrame(test['id'])
submission['Class'] = test_pred[:, 1]
submission.to_csv('submission.csv', index=False)


In [ ]:

!kaggle competitions submit -c playground-series-s3e4 -f submission.csv -m '1'